In [33]:
import torch

b = torch.range(1,12,1).reshape([3,2,2])
print(b,b.shape)

d = torch.max(b,dim=0,keepdim=True)
d.view(-1,2)

tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]]) torch.Size([3, 2, 2])


/home/huangjialong/miniconda3/envs/dist-pu/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'torch.return_types.max' object has no attribute 'view'

In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, average_precision_score,precision_score,f1_score,recall_score

# create confusion matrix
y_true = np.array([-1]*70 + [0]*160 + [1]*30)
y_pred = np.array([-1]*40 + [0]*20 + [1]*20 + 
                  [-1]*30 + [0]*80 + [1]*30 + 
                  [-1]*5 + [0]*15 + [1]*20)
cm = confusion_matrix(y_true, y_pred)
conf_matrix = pd.DataFrame(cm, index=['Cat','Dog','Pig'], columns=['Cat','Dog','Pig'])

print(y_true.shape)
print(y_pred.shape)
print('------Weighted------')
print('Weighted precision', precision_score(y_true, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_true, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_true, y_pred, average='weighted'))
print('------Macro------')
print('Macro precision', precision_score(y_true, y_pred, average='macro'))
print('Macro recall', recall_score(y_true, y_pred, average='macro'))
print('Macro f1-score', f1_score(y_true, y_pred, average='macro'))
print('------Micro------')
print('Micro precision', precision_score(y_true, y_pred, average='micro'))
print('Micro recall', recall_score(y_true, y_pred, average='micro'))
print('Micro f1-score', f1_score(y_true, y_pred, average='micro'))

(260,)
(260,)
------Weighted------
Weighted precision 0.6314062748845358
Weighted recall 0.5576923076923077
Weighted f1-score 0.575114540631782
------Macro------
Macro precision 0.5193926846100759
Macro recall 0.589781746031746
Macro f1-score 0.5233019853709507
------Micro------
Micro precision 0.5576923076923077
Micro recall 0.5576923076923077
Micro f1-score 0.5576923076923077


In [ ]:
a = [1,2,3]
b = a + [4]
b = tuple(b)
b

(1, 2, 3, 4)

: 

In [ ]:
import torch

ckp_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/mil-methods/output-model/output-test/resnet1-meanmil-ngc-customsplit/fold_0_model_best_auc.pt'
torch.load(ckp_path)
ckp_dict = torch.load(ckp_path)

In [ ]:
import csv
wsis = []
file_name = './simclr/train_label.csv'
with open(file_name, 'r') as f:
    lines = f.readlines()
    for row in lines:
        row = row.strip().split(',')
        wsis.append(row[0])
print(wsis)        


['BJXFK-LA-591455', 'c00567195', 'BJXFK-XXBA-579873', 'BJXFK-XA-575822', 'c00564281', 'BJXK-XIMEA-FFK-P-599781', 'BJXFK-YX577236', 'F22', 'BJXFK-XA576163', 'BJFFK-SCLC574253', 'C00564517', 'BJFFK-XA574864', 'BJFFK-XA573779', 'BJFFK-XA574570', 'BJXK-XIMEA-FFK-P-592670', 'BJXFK-YX575769', 'BJFFK-XA575058', 'F2', 'C00570399', 'BJXK-XIMEA-FFK-P-601186', 'BJXK-XIMEA-FFK-P-599581', 'BJXFK-XA577091', 'BJXFK-XA575834', 'BJXFK-XA583281', 'c00568834', 'F25', 'BJXFK-YX-576622', 'BJXFK-XA576692', 'BJFFK-XA571457', 'BJXFK-XA577014', 'c00569478', 'BJXFK-YX583088', 'BJXFK-XA577016', 'BJFFK-XA571327', 'C00569905', 'c00568308', 'BJFFK-SCLC572374', 'BJFFK-XA571332', 'c00567186', 'c00568523', 'BJFFK-XA572378', 'BJFFK-XA575640', 'BJXFK-YX584459', 'BJFFK-XA575740', 'BJXK-XIMEA-FFK-P-601001', 'BJXFK-XA-581051', 'BJXK-XIMEA-FFK-P-593609', 'c00568309', 'BJFFK-XA571956', 'BJFFK-XA570908', 'BJXK-XIMEA-FFK-P-602284', 'C00562488', 'BJXFK-XA579315', 'BJXK-XIMEA-FFK-P-598709', 'BJFFK-XA573780', 'BJFFK-XA571951', 'F

In [75]:
from torch.utils.data import Dataset
from torchvision.datasets import CIFAR10

from PIL import Image
import numpy as np

import os
import pickle

class CIFAR10Pair(CIFAR10):
    def __init__(self,
                 transform,
                 labeled: int = 10000,
                 unlabeled: int = 40000,
                 **kargs):
        super().__init__(**kargs)        
        self.transform = transform
        self.labeled, self.unlabeled = labeled, unlabeled
        self.targets = self._binarize_cifar10_class(self.targets)
        self.data, self.targets, self.prior = self._make_pu_label_from_binary_label(self.data, self.targets)
            
    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        img = Image.fromarray(img)  # .convert('RGB')
        imgs = [self.transform(img), self.transform(img)]
        return torch.stack(imgs), target  # stack a positive pair
    
    ## add function ##
    def _binarize_cifar10_class(self, y):
        """将类别分为animal和vehicle"""
        # 先转化为numpy
        y = np.array(y)
        y_bin = np.ones(len(y), dtype=int)
        y_bin[(y == 2) | (y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 7)] = 0
        return y_bin
    
    def _make_pu_label_from_binary_label(self, x, y):
        """挑选出一定的正样本数作为已标注标签"""
        """from https://github.com/kiryor/nnPUlearning"""
        y = np.array(y)
        labels = np.unique(y)
        positive, negative = labels[1], labels[0]
        labeled, unlabeled = self.labeled, self.unlabeled
        assert(len(x) == len(y))
        perm = np.random.permutation(len(y))
        x, y = x[perm], y[perm]
        n_p = (y == positive).sum()
        n_lp = labeled
        n_n = (y == negative).sum()
        n_u = unlabeled
        if labeled + unlabeled == len(x):
            n_up = n_p - n_lp
        elif unlabeled == len(x):
            n_up = n_p
        else:
            raise ValueError("Only support |P|+|U|=|X| or |U|=|X|.")
        _prior = float(n_up) / float(n_u)
        xlp = x[y == positive][:n_lp]
        xup = np.concatenate((x[y == positive][n_lp:], xlp), axis=0)[:n_up]
        xun = x[y == negative]
        x = np.asarray(np.concatenate((xlp, xup, xun), axis=0))
        print(x.shape)
        y = np.asarray(np.concatenate((np.ones(n_lp), -np.ones(n_u))))
        perm = np.random.permutation(len(y))
        y[y==-1]=negative
        x, y = x[perm], y[perm]
        return x, y, _prior


In [127]:

class Whole_Slide_Patchs_Ngc(Dataset):
    # pos is 0 and neg is 1, because all patches of the neg wsi are neg,
    # but pos wsi includes pos and neg patches 
    def __init__(self,
                 data_dir,
                 train_label_path,
                 transform):
        # get img_path
        sub_paths = [
            'Unannotated_KSJ/Unannotated-KSJ-TCTNGC-NILM',
            'Unannotated_KSJ/Unannotated-KSJ-TCTNGC-POS',
            'Unannotated_XIMEA/Unannotated-XIMEA-TCTNGC-NILM',
            'Unannotated_XIMEA/Unannotated-XIMEA-TCTNGC-POS'
        ]
        data_roots = list(map(lambda x: os.path.join(data_dir, x), sub_paths)) 
        wsi_dirs = []
        train_wsi_lists = []
        img_paths = []
        with open(train_label_path, 'r') as f:
            lines = f.readlines()
            for row in lines:
                row = row.strip().split(',')
                train_wsi_lists.append(row[0])
        for data_root in data_roots:
            wsi_dirs.extend([os.path.join(data_root, subdir) for subdir in os.listdir(data_root)])
        
        for wsi_path in wsi_dirs:
            wsi_name = os.path.basename(wsi_path)
            if wsi_name not in train_wsi_lists:
                continue
            img_paths.extend(glob.glob(os.path.join(wsi_path, '*.jpg')))
        self.img_paths = img_paths
        # the size is too big
        self.transform = transform
        
    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx])
        target = 0 
        if 'NILM' in str(self.img_paths[idx]):
            target = 1
        imgs =  [self.transform(img), self.transform(img)]
        return torch.stack(imgs), target
    
    def __len__(self):
        return len(self.img_paths)

    def __str__(self) -> str:
        return f'the length of patchs in {self.img_paths} is {self.__len__()}'


In [131]:
from torchvision import transforms
import glob

train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(p=0.5),
                                          transforms.ToTensor()])
    

train_set = Whole_Slide_Patchs_Ngc(
            data_dir='/home1/wsi/ngc-2023-1333/',
            train_label_path='/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/ngc_labels/ngc_train_label.csv',
            transform=train_transform
        )


In [139]:
idx = 200000
print(train_set[idx][1])
print(train_set.img_paths[idx])

0
/home1/wsi/ngc-2023-1333/Unannotated_KSJ/Unannotated-KSJ-TCTNGC-POS/c00570398/19_9.jpg


In [34]:
import torch
from torchvision import transforms

class args_class():
    def __init__(self,):
        self.data_dir = '/home/huangjialong/projects/SimCLR-CIFAR10/data'
        
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor()])
args = args_class()
train_set = CIFAR10Pair(root=args.data_dir,
                        train=True,
                        transform=train_transform,
                        download=True)    

Files already downloaded and verified


In [37]:
train_set[10][0].shape

torch.Size([2, 3, 224, 224])

In [36]:
train_set[0][1]

6

In [ ]:
import torch

tensor = torch.tensor([1.0, -2.0, 3.0, -4.0])
sign_tensor = torch.sign(tensor)

print(sign_tensor)  # 输出: tensor([1., -1., 1., -1.])

tensor([ 1., -1.,  1., -1.])
